In [1]:
# Copyright 2022 Google LLC
# Authors: 
# Fabian Hirschmann <fhirschmann@google.com>, 
# Elia Secchi <eliasecchi@google.com>,
# Megha Agarwal <meghaag@google.com>,
# Mandie Quartly <mandieq@google.com>
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Automated MLOps pipeline build, testing and deployment

In the previous notebook, you created a machine learning pipeline to train a model. In this session, it's all about automating the training and deployment of this model. Hence, the objective this notebook is to:

1. Refactor your Kubeflow pipeline into a Python file that can be compiled into YAML in an automated fashion.
1. Write a script to deploy a compiled Kubeflow pipeline to Vertex AI.
1. Use Cloud Build (CI/CD) to compile, test, and run your Kubeflow pipeline.
1. Create a Cloud Source Repository (Git) to automatically trigger Cloud Build on every change on the master branch
1. [Optional] Create a [pipeline template](https://cloud.google.com/vertex-ai/docs/pipelines/create-pipeline-template) to allow for the pipeline to be reused and retriggered
1. [Optional] Setup a schedule for the pipeline, which can be done in 2 ways:
      - using Cloud Scheduler job, which sends a message to a Pub/Sub topic, and then calls a Cloud Function to trigger the VertexAI pipeline.
      - using the new Vertex AI Pipelines Schedules API
      
      
As part of this notebook, you'll create the following files using the `%%writefile` directive.
- `src/requirements.txt`: Python requirements file listing all dependencies.
- `src/pipeline.py`: File containing your Kubeflow pipeline in Python and logic to compile the pipeline into YAML.
- `src/create-pipeline-template.py`: Python script to create a Kubeflow Pipeline Template in Vertex AI.
- `src/submit-pipeline.py`: Python script to submit pipeline to Vertex AI.
- `src/cloudbuild.yml`: Cloud build pipeline to run your CI/CD process.
- `src/tests/test_pipeline.py`: Unit test for pipeline
- `cf-trigger/main.py`: Cloud Function to trigger your Kubeflow Pipeline.

## Prerequisites

Ensure your Compute Engine default service account `{PROJECT_NUMBER}-compute@developer.gserviceaccount.com` has following permissions enabled:
1. `Vertex AI User`
1. `Cloud Build Editor`
1. `Artifact Registry Writer`
1. `Source Repository Administrator`
1. `Storage Object Creator`
1. `Storage Object Viewer`

Ensure your Cloud Build default service account `{PROJECT_NUMBER}-@cloudbuild.gserviceaccount.com` has following permissions enabled:
1. `Service Account User`
1. `Vertex AI User`

Please note relevant IAM permissions can sometimes take time (up to 15 min) to trickle through.

In [ ]:
!mkdir -p src

In [ ]:
%%writefile src/requirements.txt
kfp==2.0.0b9
pytest==7.2.0
pytz==2022.7
google-cloud-aiplatform==1.20.0
google-api-core==2.10.2
google-auth==1.35.0
google-cloud-bigquery==1.20.0
google-cloud-core==1.7.3
google-cloud-resource-manager==1.6.3
google-cloud-storage==2.2.1


In [ ]:
!pip install -r src/requirements.txt --user

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
     
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

!gcloud config set project {PROJECT_ID}

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"
    
BUCKET_NAME = f"mlops-coaching-2-{PROJECT_ID}"
EXPERIMENT_NAME = "mlops-coaching-2-experiment"
PIPELINE_NAME = "mlops-coaching-2-pipeline"
ENDPOINT_NAME = "mlops-coaching-2-endpoint"
REPOSITORY_NAME = f"mlops-coaching-2-{PROJECT_ID}"

In [ ]:
!gsutil mb -c regional -l $REGION gs://$BUCKET_NAME

Using the cell below, ensure the following cloud service APIs are enabled for this lab:
1. `Vertex AI API`
1. `Cloud Build API`
1. `Artifact Registry API`
1. `Cloud Source Repositories API`

In [ ]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable artifactregistry.googleapis.com
!gcloud services enable sourcerepo.googleapis.com

## Automated MLOps pipeline creation

### Create a script containing your Vertex AI/Kubeflow Pipeline to compile the pipeline into `pipeline.yaml`

> <font color='green'>**Task 1**</font>
>
> Create a Python script `src/pipeline.py` that creates a file name `pipeline.yaml` from the Kubeflow pipeline you developed last week. The output file should be in YAML and not JSON format.
>
> If you were unable to produce a Kubeflow pipeline last week, please use the one provided below. Otherwise, replace it with your own.

In [ ]:
%%writefile src/pipeline.py

## Your code goes below this line

#### TODO: INSERT PIPELINE FROM MLOPS PART 1 ####

from typing import NamedTuple

from kfp.dsl import pipeline
from kfp.dsl import component
from kfp import compiler

@component() 
def concat(a: str, b: str) -> str:
    return a + b

@component
def reverse(a: str) -> NamedTuple("outputs", [("before", str), ("after", str)]):
    return a, a[::-1]

@pipeline(name="mlops-coaching-pipeline")
def basic_pipeline(a: str='stres', b: str='sed'):
    concat_task = concat(a=a, b=b)
    reverse_task = reverse(a=concat_task.output)

if __name__ == '__main__':
    compiler.Compiler().compile(pipeline_func=basic_pipeline, package_path="pipeline.yaml")

Using the next command, you can test the materialized pipeline generated by your script. You can view the output in a file named `pipeline.yaml`.

In [ ]:
!python src/pipeline.py

In [ ]:
!head -n20 pipeline.yaml

### Test the Pipeline

> <font color='green'>**Task 2**</font>
> Write unit/integration tests for the pipeline you created to ensure the component logic that you added works as expected

In [ ]:
!mkdir -p src/tests

In [ ]:
%%writefile src/tests/test_pipeline.py

import unittest
from pipeline import concat, reverse, basic_pipeline

class TestBasicPipeline(unittest.TestCase):
    # def setUp(self):
        # Get relevant component
    
    def test_concat_component(self):
        self.assertEqual(concat.python_func(3, 3), 6)

    def test_reverse(self):
        self.assertEqual(reverse.python_func("stressed")[1], "desserts")

    def test_pipeline(self):
        pass

if __name__ == '__main__':
    unittest.main()

Using the next command, you can run the tests in the script using python `unittest` test runner. It discovers all the test files that start with `test_*`

You can also use other testing framework of your choice (e.g. `pytest`)

In [ ]:
!PYTHONPATH=src python -m unittest discover -s src/tests/

### Create a script to submit your compile kubeflow pipeline (`pipeline.yaml`) to Vertex AI

In [ ]:
%%writefile src/submit-pipeline.py
import os

from google.cloud import aiplatform
import google.auth

PROJECT_ID = os.getenv("PROJECT_ID")
if not PROJECT_ID:
    creds, PROJECT_ID = google.auth.default()

REGION = os.environ["REGION"]
BUCKET_NAME = os.environ["BUCKET_NAME"]
EXPERIMENT_NAME = os.environ["EXPERIMENT_NAME"]
ENDPOINT_NAME = os.environ["ENDPOINT_NAME"]
PIPELINE_NAME = os.environ["PIPELINE_NAME"]

aiplatform.init(project=PROJECT_ID, location=REGION)
sync_pipeline = os.getenv("SUBMIT_PIPELINE_SYNC", 'False').lower() in ('true', '1', 't')

job = aiplatform.PipelineJob(
    display_name=PIPELINE_NAME,
    template_path='pipeline.yaml',
    location=REGION,
    project=PROJECT_ID,
    enable_caching=True,
    pipeline_root=f'gs://{BUCKET_NAME}'
)
print(f"Submitting pipeline {PIPELINE_NAME} in experiment {EXPERIMENT_NAME}.")
job.submit(experiment=EXPERIMENT_NAME)

if sync_pipeline:
    job.wait()

Let's test this script in the Notebook. You can check the pipeline's status by clicking on the link printed by the script.

In [ ]:
%set_env REGION=$REGION
%set_env BUCKET_NAME=$BUCKET_NAME
%set_env EXPERIMENT_NAME=$EXPERIMENT_NAME
%set_env PIPELINE_NAME=$PIPELINE_NAME
%set_env ENDPOINT_NAME=$ENDPOINT_NAME
%set_env SUBMIT_PIPELINE_SYNC=1

!python src/submit-pipeline.py

### Create a pipeline template in Artifact Registry from `pipeline.yaml`

A pipeline template is a resource that you can use to publish a workflow definition so that it can be reused multiple times, by a single user or by multiple users. This feature is [documented here](https://cloud.google.com/vertex-ai/docs/pipelines/create-pipeline-template).

The Kubeflow Pipelines SDK registry client is a new client interface that you can use with a compatible registry server, such as Artifact Registry, for version control of your Kubeflow Pipelines (KFP) templates.

> <font color='green'>**Task 4**</font>
>
> Create a Python script `src/create-pipeline-template.py` that uploads `pipeline.yaml` to the Vertex AI Pipeline registry.
>

In [ ]:
!gcloud artifacts repositories create mlops2-repo --location=$REGION --repository-format=KFP

In [ ]:
%%writefile src/create-pipeline-template.py
import os
import google.auth

from kfp.registry import RegistryClient

PROJECT_ID = os.getenv("PROJECT_ID")
if not PROJECT_ID:
    creds, PROJECT_ID = google.auth.default()
REGION = os.environ["REGION"]

## Your code goes below this line

client = RegistryClient(host=f"https://{REGION}-kfp.pkg.dev/{PROJECT_ID}/mlops2-repo")

template_name, template_version = client.upload_pipeline(
  file_name="pipeline.yaml",
  tags=["v1", "latest"]
)

In [ ]:
!python src/create-pipeline-template.py

### Automate Kubeflow pipeline compilation, template generation, and execution through Cloud Build

Cloud Build is a service that executes your builds on Google Cloud. In this exercise, we want to use it to both compile and run your machine learning pipeline. For more information, please refer to the [Cloud Build documentation](https://cloud.google.com/build/docs/overview).

In [ ]:
%%writefile src/cloudbuild.yaml
steps:
  # Install dependencies
  - name: 'python'
    entrypoint: 'pip'
    args: ["install", "-r", "requirements.txt", "--user"]

  # Compile pipeline
  - name: 'python'
    entrypoint: 'python'
    args: ['pipeline.py']
    id: 'compile'

  # Test the Pipeline Components 
  - name: 'python'
    entrypoint: 'python'
    args: ['-m', 'unittest', 'discover', 'tests/']
    id: 'test_pipeline'
    waitFor: ['compile']

  # Upload compiled pipeline to GCS
  - name: 'gcr.io/cloud-builders/gsutil'
    args: ['cp', 'pipeline.yaml', 'gs://${_BUCKET_NAME}']
    id: 'upload'
    waitFor: ['test_pipeline']
        
  # Run the Vertex AI Pipeline (synchronously for test/qa environment).
  - name: 'python'
    id: 'test'
    entrypoint: 'python'
    env: ['BUCKET_NAME=${_BUCKET_NAME}', 'EXPERIMENT_NAME=qa-${_EXPERIMENT_NAME}', 'PIPELINE_NAME=${_PIPELINE_NAME}',
          'REGION=${_REGION}', 'ENDPOINT_NAME=qa-${_ENDPOINT_NAME}', 'SUBMIT_PIPELINE_SYNC=true']
    args: ['submit-pipeline.py']

  # Create pipeline template and upload it to the artifact registry
  - name: 'python'
    id: 'template'
    entrypoint: 'python'
    env: ['REGION=${_REGION}']
    args: ['create-pipeline-template.py']
    
  # Run the Vertex AI Pipeline (asynchronously for prod environment). In a real production scenario, this would run in a different GCP project.
  - name: 'python'
    id: 'prod'
    entrypoint: 'python'
    env: ['BUCKET_NAME=${_BUCKET_NAME}', 'EXPERIMENT_NAME=prod-${_EXPERIMENT_NAME}', 'PIPELINE_NAME=${_PIPELINE_NAME}',
          'REGION=${_REGION}', 'ENDPOINT_NAME=prod-${_ENDPOINT_NAME}', 'SUBMIT_PIPELINE_SYNC=false']
    args: ['submit-pipeline.py']

Cloud Build uses a special service account to execute builds on your behalf. When you enable the Cloud Build API on a Google Cloud project, the Cloud Build service account is automatically created and granted the Cloud Build Service Account role for the project. This role gives the service account permissions to perform several tasks, however you can grant more permissions to the service account to perform additional tasks. [This page](https://cloud.google.com/build/docs/securing-builds/configure-access-for-cloud-build-service-account) explains how to grant and revoke permissions to the Cloud Build service account.

For Cloud Build to be able to deploy your pipeline, you need to give its' service account `{PROJECT_NUMBER}@cloudbuild.gserviceaccount.com` the **Vertex AI User** and **Service Account User** role. Note that it may take up to 5 minutes until the new permissions propagate.

In [ ]:
!gcloud builds submit ./src --config=src/cloudbuild.yaml --substitutions=_BUCKET_NAME=$BUCKET_NAME,_EXPERIMENT_NAME=$EXPERIMENT_NAME,_PIPELINE_NAME=$PIPELINE_NAME,_REGION=$REGION,_ENDPOINT_NAME=$ENDPOINT_NAME

### Create a git repository and trigger Cloud Build execution

Before you can create the repository here, please **enable Cloud Source Repositories API** in the Google Cloud console.

> <font color='green'>**Task 5**</font>
>
> Create a build trigger on the source repository that executes `cloudbuild.yaml`. Make sure to pass all **--substitutions**.
>

In [ ]:
!gcloud source repos create $REPOSITORY_NAME

In [ ]:
## Your code goes below this line

!gcloud beta builds triggers create cloud-source-repositories \
    --name=mlops2-source-trigger \
    --repo=$REPOSITORY_NAME \
    --branch-pattern=master \
    --build-config=cloudbuild.yaml \
    --substitutions=_BUCKET_NAME=$BUCKET_NAME,_EXPERIMENT_NAME=$EXPERIMENT_NAME,_PIPELINE_NAME=$PIPELINE_NAME,_REGION=$REGION,_ENDPOINT_NAME=$ENDPOINT_NAME

In [ ]:
!gcloud source repos clone $REPOSITORY_NAME --project=$PROJECT_ID

In [ ]:
!cp -av src/* $REPOSITORY_NAME/

In [ ]:
!cd $REPOSITORY_NAME && git add .

In [ ]:
!git config --global user.email "{YOUR_EMAIl}"
!git config --global user.name "{YOUR_NAME}"

In [ ]:
!cd $REPOSITORY_NAME && git commit -a -m "initial commit"

In [ ]:
!cd $REPOSITORY_NAME && git push -u origin master

## Schedule pipeline execution

> <font color='green'>**Task 6**</font>
>
> Schedule the execution of the pipeline such that it runs every day.
>

### Schedule Method 1: Cloud Scheduler → Pub/Sub → Cloud Functions

Using this method, you create a Pub/Sub topic that triggers a Cloud Function that triggers a Vertex AI Pipeline. Note that the service account the Cloud Function runs as needs access to Cloud Storage and Vertex AI.

In [ ]:
!gcloud pubsub topics create trigger-mlops-coaching-2-pipeline

In [ ]:
!mkdir -p cf-trigger

In [ ]:
%%writefile cf-trigger/requirements.txt
kfp==2.0.0b9
pytest==7.2.0
google-cloud-aiplatform==1.20.0
pytz==2022.7

In [ ]:
%%writefile cf-trigger/main.py

## Your code goes below this line

import os
import base64
import json
from google.cloud import aiplatform

REGION = os.environ["REGION"]
PIPELINE_NAME = os.environ["PIPELINE_NAME"]
PROJECT_ID = os.environ["PROJECT_ID"]
BUCKET_NAME = os.environ["BUCKET_NAME"]


def subscribe(event, context):
    aiplatform.init(project=PROJECT_ID, location=REGION)
    
    job = aiplatform.PipelineJob(
        display_name=PIPELINE_NAME,
        template_path=f'gs://{BUCKET_NAME}/pipeline.yaml',
        location=REGION,
        project=PROJECT_ID,
        enable_caching=False,
        pipeline_root=f'gs://{BUCKET_NAME}'
    )

    job.submit()

In [ ]:
## Your code goes below this line

!gcloud functions deploy mlops-coaching-2-function \
--source=./cf-trigger \
--entry-point=subscribe \
--trigger-topic trigger-mlops-coaching-2-pipeline \
--runtime python37 \
--ingress-settings internal-and-gclb \
--set-env-vars REGION=$REGION,PIPELINE_NAME=$PIPELINE_NAME,PROJECT_ID=$PROJECT_ID,BUCKET_NAME=$BUCKET_NAME

In [ ]:
## Your code goes below this line

# TODO

!gcloud scheduler jobs create pubsub

### Schedule Method 2: Vertex AI Scheduler (in preview)

To use the Schedules API during the Private Preview release, your project ID must be allowlisted in the `SCHEDULED_RUNS_TRUSTED_TESTER` group, you can use projects that you previously signed up with—for example. If you’re not in the trusted testers group, sign up by using this [sign-up form](https://docs.google.com/forms/d/e/1FAIpQLScDxABxIvqjeM_279dwTMmVfFBJD7qmW2leyU_ZBTYutJ62uA/viewform?usp=sf_link).

The Vertex AI Schedule Service API is a new resource that lets you schedule ad hoc or recurring Vertex AI Pipeline runs. The goal of this service is to make scheduling a one off or recurring pipeline run simple, such that any Vertex AI user can quickly understand and leverage schedules to implement naive continuous training in their business.

This new API will replace our previous guidance of using Cloud Scheduler with Cloud Functions to schedule a pipeline run. It may support additional Vertex AI resources in the future.

Note that **Scheduler is currently only available via its REST interface and not the Vertex AI SDK**.

In [ ]:
from google.cloud import aiplatform
import json

job = aiplatform.PipelineJob(
    display_name=PIPELINE_NAME,
    template_path='pipeline.yaml',
    location=REGION,
    project=PROJECT_ID,
    pipeline_root=f'gs://{BUCKET_NAME}'
)

In [ ]:
PIPELINE_TEMPLATE = job.to_dict()

In [ ]:
import requests
import google.auth
import google.auth.transport.requests

creds, project = google.auth.default()
auth_req = google.auth.transport.requests.Request()
creds.refresh(auth_req)
token = creds.token
service_account_email = creds.service_account_email

In [ ]:
endpoint = f"https://{REGION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{REGION}/schedules"
headers = {"Authorization": f"Bearer {creds.token}"}

## Your code goes below this line

payload = {
    "display_name": "mlops2-schedule",
    "cron": "0 1 * * *",
    "max_concurrent_run_count": "2",
}

print(requests.post(endpoint, json=payload, headers=headers).json())